In [59]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

import pymysql
pymysql.install_as_MySQLdb()

import datetime as dt
#from sqlalchemy import desc

In [2]:
start = '2015-04-15'
end = '2015-05-01'

**Create engine**

In [3]:
engine = create_engine('sqlite:///../Base/hawaii.sqlite')

In [4]:
#check if everything works
engine.execute('select * from station').fetchall()

[(0, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (1, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (2, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (3, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (4, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (5, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (6, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9),
 (7, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (8, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

**Check tables and column names**

In [15]:
inspector = inspect(engine)

In [33]:
tables = {} # all tables and columns. maybe useless
for table in inspector.get_table_names():
    print(f'Table name is "{table}", it contains columns: ')
    columns = inspector.get_columns(table)
    names = []
    for c in columns:
        names.append(c['name'])
        print(c['name'], c['type'])
    print()
    tables[table] = names

Table name is "measurement", it contains columns: 
index INTEGER
station VARCHAR
date VARCHAR
prcp FLOAT
tobs INTEGER

Table name is "station", it contains columns: 
index INTEGER
station VARCHAR
name VARCHAR
latitude FLOAT
longitude FLOAT
elevation FLOAT



**Declare a Base using automap and reflect database**

In [48]:
Base = automap_base()
Base.prepare(engine, reflect=True)

**Assign classes to variable**

In [49]:
Measurement = Base.classes.measurement
Station = Base.classes.station

***Precipitation Analysis***

**Start a session to make queries**

In [50]:
session = Session(engine)

In [90]:
#find the last date
last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
print(last_date[0])

2017-08-23


In [74]:
year_ago = last_date[0] - dt.timedelta(days=365)

TypeError: unsupported operand type(s) for -: 'str' and 'datetime.timedelta'

In [73]:
last_year = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date == last_date[0]).all()

In [ ]:
for row in session.query(Measurement.date == '2015-01-03', Measurement.prcp).limit(5).all():
    print(row)

In [ ]:
session.query(Measurement).group_by(Measurement.station).count()

In [ ]:
session.query(Measurement, Measurement.date).order_by(desc(Measurement.date)).first()

In [ ]:
prec_year = session.query(Measurement).filter(Measurement.date > '2016-08-23').all()

In [ ]:
test[3]